# Fine-Tuning StableDiffusion XL with DreamBooth

Over the past few years Generative AI models have popped up everywhere - from creating realistic responses to complex questions, to generating images and music to impress art critics around the globe. In this notebook we use the Hugging Face [Stable Diffusion XL (SDXL)](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) model to create images from text prompts. You'll see how to import the SDXL model and use it to generate an image. 

From there, you'll see how you can fine-tune the model using [DreamBooth](https://huggingface.co/docs/diffusers/training/dreambooth), a method for easily fine-tuning a text-to-image model. We'll use a small number of photos of **my dog** in this notebook to fine-tune SDXL. This will allow us to generate new images that include **my dog**! 

**IMPORTANT:** This project will utilize additional third-party open source software. Review the license terms of these open source projects before use. Third party components used as part of this project are subject to their separate legal notices or terms that accompany the components. You are responsible for confirming compliance with third-party component license terms and requirements.

### Stable Diffusion XL Model

First, we import the classes and libraries we need to run the notebook.

In [2]:
# !python -m venv nvidia-usecase
!source nvidia-usecase/bin/activate

!pip install -q typing python-dotenv minio

output_model_dir = "../models/tuned-toy-jensen/model"

Now we can use Hugging Face and DreamBooth to fine-tune this model. To do this we create a config, then specify some flags like an instance prompt, a resolution and a number of training steps for the fine-tuning algorithm to run. 

In [17]:
from minio import Minio
from minio.error import S3Error
from dotenv import load_dotenv
from typing import NamedTuple
import os
import pathlib

class S3Env(NamedTuple):
  access_key_id: str
  secret_access_key: str
  s3_endpoint: str
  bucket_name: str
  default_region: str

class MinioBucketMeta(NamedTuple):
  client: Minio
  bucket_name: str
  prefix: str = ""
  model_data_dir: str = None
  file_name: str = None
  object_name: str = None

In [18]:
def init_env() -> S3Env:
  load_dotenv()

  access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
  secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
  default_region = os.environ.get('AWS_DEFAULT_REGION')
  s3_endpoint = os.environ.get('AWS_S3_ENDPOINT')
  bucket_name = os.environ.get('AWS_S3_BUCKET')

  # print(f'key_id={access_key_id}, secret_key={secret_access_key}')
  # print(f's3_endpoint={s3_endpoint}, bucket={bucket_name}')

  return S3Env(access_key_id, secret_access_key, s3_endpoint, bucket_name, default_region)

def init_minio(s3_env: S3Env, isSecure: bool = True) -> Minio:
  return Minio(
    s3_env.s3_endpoint,
    access_key = s3_env.access_key_id,
    secret_key = s3_env.secret_access_key,
    secure = isSecure
  )

def list_objects_minio(bucket_name: str, client: Minio):
  print(f'\n\nListing all the object in the bucket [{bucket_name}] from Minio')

  try:
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
      print(f'name={obj.object_name}, modified={obj.last_modified}, etag={obj.etag}, size={obj.size}, content_type={obj.content_type}')
  except S3Error as e:
    print("Error occurred: ", e)

def upload_file(file_model: MinioBucketMeta):
  client: Minio = file_model.client

  # Replace the model directory name with the prefix (for the bucket object)
  object_name = file_model.file_name.replace(file_model.model_data_dir, file_model.prefix)

  print(f'   -> Trying to upload file [{file_model.file_name}] with key [{object_name}] to the bucket [{file_model.bucket_name}]...', end=" ")

  try:
    client.fput_object(file_model.bucket_name, object_name, file_model.file_name)
    print('SUCCESS')
  except S3Error as e:
    print('FAILURE')
    print("Error:: ", e)


def upload_files(dir_model: MinioBucketMeta):
  print(f'Uploading files from [{dir_model.model_data_dir}] directory...')
  client = dir_model.client

  if not client.bucket_exists(dir_model.bucket_name):
    client.make_bucket(dir_model.bucket_name)
    print(f' -->> Created bucket [{dir_model.bucket_name}]')
  else:
    print(f' -->> Bucket already exists [{dir_model.bucket_name}]')

  for (dir_path, dirs, files) in os.walk(dir_model.model_data_dir):
    print(f'root={dir_path}, dirs={dirs}, files={files}')
    for f in files:
      if not f.startswith('.'):
        file_name = pathlib.Path(dir_path, f)

        upload_file(MinioBucketMeta(client=dir_model.client,
                                          bucket_name=dir_model.bucket_name,
                                          file_name=str(file_name),
                                          prefix=dir_model.prefix,
                                          model_data_dir=dir_model.model_data_dir))
      else:
        print(f'   -->> *** File (or dir) [{f}] IGNORED... *** <<--')  